# Preprocessing

Ze względu na występowanie w pliku godzin, które są nie zgodne z ogólnie przyjętym formatem (autobusy noce), przygotowałem funkcję, która przy wczytywaniu danych z pliku służyć będzie zmianie godziny na prawidłową.

In [2]:
from datetime import datetime, timedelta

def read_time(time: str):
    if(int(time[:2]) > 23):
        time = str(int(time[:2])-24) + time[2:]
    return datetime.strptime(time, '%H:%M:%S')
                

# Struktura danych

Klasa Node przechowywać będzie informacje dotyczące pojedynczego węzła(przystanku autobusowego) występującego w grafie. Każdy przystanek posiada określona nazwę długosć geograficzną, szerokość geograficzną oraz listę wychodzących z niego krawędzi. Dodatkowo zdefiniowane zostały w nim funkcję mające służyć jako pomoc przy wczytywaniu danych. Fix_location to funkcja, która w przypadu gdy podana w argumentach szerokość lub długość geograficzna nie jest identyczna, co aktualnie zapisana, oblicza i przypisuje średnią wyciągniętą z obu wartości. Funkcja add_edge korzystając z podanych argumentów tworzy obiekt Edge i dodaje go do listy krawędzi.

In [3]:
class Node:
    def __init__(self, stop_name, lat, lon):
        self.stop_name = stop_name
        self.lat = lat
        self.lon = lon
        self.edges = []

    def add_edge(self, line, start_stop, end_stop, departure_time, arrival_time):
        self.edges.append(Edge(line, start_stop, end_stop, departure_time, arrival_time))
        
    def fix_lat(self, lat):
        if(self.lat!=lat):
            self.lat=(self.lat+lat)/2
            
    def fix_lon(self, lon):
        if(self.lon!=lon):
            self.lon=(self.lon+lon)/2 
            
    def fix_location(self, lat, lon):
        self.fix_lat(lat)
        self.fix_lon(lon)

    def __lt__(self, other):
        return self.stop_name <= other.stop_name

    def __eq__(self, other):
        return self.stop_name == other.stop_name

    def __ne__(self, other):
        return self.stop_name != other.stop_name

    def __str__(self) -> str:
        edges_str = '\n'.join(str(edge) for edge in self.edges)
        return f'stop_name: {self.stop_name}\n edges: {edges_str}'


Klasa Edge służyć będzie do przechowywania informacji o krawędzi między węzłami (połączenie między przystankami). Każde połączenie posiada nazwę linii, przystanek początkowy, przystanek końcowy, czas odjazdu z obecnego przystanku oraz czas przyjazdu na przystanek docelowy.

In [4]:

class Edge:
    def __init__(self, line, start_stop, end_stop, departure_time, arrival_time):
        self.line = line
        self.start_stop = start_stop
        self.end_stop = end_stop
        self.departure_time = departure_time
        self.arrival_time = arrival_time
        
    def __str__(self):
        return f'{self.line} : {self.start_stop.stop_name} - {self.end_stop.stop_name} | {self.departure_time} :: {self.arrival_time}'


    def __lt__(self, other):
        return self.arrival_time <= other.arrival_time

# Wczytywanie danych do struktury

Cała struktura grafu będzie przechowywana w słowniku. Kluczem będzie nazwa przystanku, natomiast wartością obiekt węzła. W związku z niespójnym nazewnictwem wszystkie nazwy przystanków przechowywane będą wielkimi literami. W przypadku odczytania wiersza w pliku, gdzie przystanek startowy nie występuje w słowniku tworzę obiekt, który ma go reprezentować, natomiast gdy już został wcześniej dodany do struktury wywołuję funkcję fix_location, która ma służyć uśrednieniu jego długości i szerokości geograficznej. Następnie do węzła przekazywane są dane dotyczące odczytanej krawędzi, aby móc dodać go do listy wychodzących z niego połączeń. Po wczytaniu do słownika całego pliku lista krawędzi wychodzących z węzłów sortowana jest według czasu odjazdu, co ma służyć ułatwieniu przyszłego debugowania.

In [5]:
import csv

def load_data_to_graph(csv_file_path):
    nodes_dict = {}

    with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)

        for row in reader:
            line = row['line']
            departure_time = read_time(row['departure_time'])
            arrival_time = read_time(row['arrival_time'])
            start_stop_name = (row['start_stop']).upper()
            end_stop_name = (row['end_stop']).upper()
            start_stop_lat = float(row['start_stop_lat'])
            start_stop_lon = float(row['start_stop_lon'])
            end_stop_lat = float(row['end_stop_lat'])
            end_stop_lon = float(row['end_stop_lon'])

            if start_stop_name not in nodes_dict:
                nodes_dict[start_stop_name] = Node(start_stop_name, start_stop_lat, start_stop_lon)
            else:
                nodes_dict[start_stop_name].fix_location(start_stop_lat, start_stop_lon)

            if end_stop_name not in nodes_dict:
                nodes_dict[end_stop_name] = Node(end_stop_name, end_stop_lat, end_stop_lon)
            else:
                nodes_dict[end_stop_name].fix_location(end_stop_lat, end_stop_lon)
                
            
            nodes_dict[start_stop_name].add_edge(line, nodes_dict[start_stop_name],nodes_dict[end_stop_name], departure_time, arrival_time)

    for node_name, node in nodes_dict.items():
        node.edges.sort(key=lambda x: x.departure_time)

    return nodes_dict

In [6]:
csv_file = 'connection_graph.csv'
graph = load_data_to_graph(csv_file)

# Kryterium czasu

Funkcje pomocnicze

Na początku zdefiniowane zostaną funkcje służące obliczeniu kosztów przejazdu. Funkcja calculate_timediff jest kluczowa dla działania wszytskich algorytmów, gdyż oblicza różnicę między czasem aktualnym, a czasem dojazdu na następny przystanek, uwzględniając sytuację, gdy czas odjazdu lub przyjazdu na przystanek docelowy jest mniejszy niż czas obecny (czyli wtedy gdy połączenie następuje kolejnego dnia). 

In [7]:
def calculate_timediff(departure_time, edge_departure_time, edge_arrival_time):
    if(edge_departure_time < departure_time or edge_arrival_time < departure_time):
        midnight = datetime.strptime('00:00:00', "%H:%M:%S")
        duration = edge_arrival_time - midnight
        midnight += timedelta(days=1)
        duration += midnight - departure_time
    else:
        duration = edge_arrival_time - departure_time
        
    return duration.total_seconds() / 60.0

In [8]:
def sum_cost(current_cost, departure_time, edge_departure_time, edge_arrival_time):
    return current_cost + calculate_timediff(departure_time, edge_departure_time, edge_arrival_time)

Dijkstra

Koszt dojazdu na każdy z przystanków przechowywany będzie w słowniku cost_dict, a trasa w liście path. Na początku dojazd na każdy z przystanków oprócz początkowego wynosić będzie 99999999, natomiast trasa będzie pusta. Tworzymy strukturę heapq, która służyć będzie jako kolejka priorytetowa, gdzie kryterium pierszeństwa będzie najmniejszy koszt(czas) przejazdu. Pierwszym elementem dodanym do heapq jest krotka z kosztem 0, czasem początkowym, obiektem przystanku początkowego oraz ścieżką. Następnie w pętli wykonującej się dopóki kolejka priorytetowa nie jest pusta lub obecny węzeł nie jest węzłem końcowym zczytujemy z kolejki priorytetowej dotychczasowy koszt, godzinę, przystanek oraz ścieżkę. Teraz dla każdej z krawędzi wychodzącej z obecnego węzła obliczamy koszt dotarcia na przystanek do którego prowadzi połączenie. Jeżeli czas obecnie przechowywany w słowniku kosztu dla następnego przystanku będzie mniejszy niż nowo wylicznony, to zosatnie on zastąpiony, a do kolejki dodany zostanie element zawierający nowy koszt, czas dojazdu na nastepny przystanek, obiekt następnego przystanku oraz ścieżka uaktualniona o rozważane połączenie.

In [9]:
import heapq
import time

def dijkstra_time(nodes_dict, start_stop, end_stop, start_time):
    cost_dict={}

    for node_name in nodes_dict.keys():
        cost_dict[node_name] = 99999999

    cost_dict[start_stop] = 0

    path = []

    priority_queue = [(0, start_time, nodes_dict[start_stop], path)]

    while priority_queue:
        (current_cost, current_arrival_time, current_node, current_path) = heapq.heappop(priority_queue)

        if(current_node == nodes_dict[end_stop]):
            path = current_path
            break

        for edge in current_node.edges:
            
            next_node = edge.end_stop
            new_cost = sum_cost(current_cost, current_arrival_time, edge.departure_time, edge.arrival_time)
            
            if(new_cost < cost_dict[next_node.stop_name]):
                cost_dict[next_node.stop_name] = new_cost

                new_path = current_path.copy()
                new_path.append(edge)

                heapq.heappush(priority_queue, (new_cost, edge.arrival_time, next_node, new_path))

               

    return path, cost_dict[end_stop]

A* - funkcje pomocnicze

Do wykonania algorytmu a* konieczne będzie stworzenie funkcji, która korzystając z długości i szerokości geograficznej zwróci nam odległość między przystankami w formie, która może być dodana do kosztów czasu. To realizować ma funkcja calculate_dist_as_time, która wywołuje funkcję obliczającą dystans euklidesowy między węzłami. Następnie obliczona odległość mnożona jest przez 416.25, co w przybliżeniu ma konwertować odległość geograficzną na czas przejazdu między przystankami. Wartość 416.25 uzyskałem porzystając ze wzoru [111 / 16 * 60], gdzie 111 - przybliżenie 1 stopnia geograficznego w km, 16 - średnia prędkość tramwaju we Wrocławiu, 60 - przeliczenie czasu z godzin na minuty.

In [10]:
def euclidian_distance(start_node: Node, end_node: Node):
    return ((start_node.lat - end_node.lat) ** 2 + (start_node.lon - end_node.lon) ** 2) ** 0.5

def manhattan_distance(start_node, end_node):
    return abs(start_node.lat - end_node.lat) + abs(start_node.lon - end_node.lon)

def calculate_dist_as_time(start_node: Node, end_node: Node):
    distance =euclidian_distance(start_node, end_node)    
    return distance * 416.25


A* - czas

Algorytm bardzo zbliżony do wcześniej opisanego algorytmu Dijskstry. Główną różnicą jest przechowywania w kolejce priorytetowej priorytetu, który jest również głównym kryterium pierszeństwa. Jego wartość wyliczana jest jako nowy koszt plus przewidywany czas dojazdu do przystanku końcowego, który obliczany jest wykorzystując funkcję pomocniczą calculate_dist_as_time.

In [11]:
def a_star_time(nodes_dict, start_stop, end_stop, start_time):
    cost_dict={}

    for node_name in nodes_dict.keys():
        cost_dict[node_name] = 99999999

    cost_dict[start_stop] = 0

    path = []

    priority_queue = [(0, 0, start_time, nodes_dict[start_stop], path)]

    while priority_queue:
        (priority, current_cost, current_arrival_time, current_node, current_path) = heapq.heappop(priority_queue)

        if(current_node == nodes_dict[end_stop]):
            path = current_path
            break

        for edge in current_node.edges:

            next_node = edge.end_stop
            new_cost = sum_cost(current_cost, current_arrival_time, edge.departure_time, edge.arrival_time)
    
            if(new_cost < cost_dict[next_node.stop_name]):
                cost_dict[next_node.stop_name] = new_cost
                
                new_priority = new_cost + calculate_dist_as_time(nodes_dict[next_node.stop_name], nodes_dict[end_stop])

                new_path = current_path.copy()
                new_path.append(edge)

                heapq.heappush(priority_queue, (new_priority, new_cost, edge.arrival_time, next_node, new_path))

    return path, cost_dict[end_stop]




Funkcje pomocnicze - kryterium przesiadek

Przy obliczaniu kryterium przysiadek zdefiniowałem 2 funkcje pomocnicze: calculate_transfer - zwracającą 1 jeżeli linia połączenia jest inna niż obecna oraz calculate_transfer_as_time, która przelicza każdą przesiadkę na 1440 min (jedną dobę).

In [12]:
def calculate_transfer(current_line, nextEdge):
    if(current_line != nextEdge.line ):
        return 1.0
    
    return 0.0

In [13]:
def calculate_transfer_as_time(transfer):
    return transfer*1440

Dijkstra - przesiadki

Kryterium przesiadek zrozumiane i zrealizowane zostało przeze mnie jako kryterium czasu z minimalną liczbą zmian linii. W związku z czym algorytm jest zbliżony do algorytmu dla kryterium samego czasu. Ważne zmiany to:
- na samym początku dla pierwszego przystanku, dla każdej linii z niego wychodzącej koszt ustawiany jest na 0.
- klucz w słowniku kosztu nie jest już nazwą przystanku, a nazwą przystanku oraz aktualnej linii. Brak dodania do klucza obecnej linii powodowałby, że do każdego przystanku po drodze znajdowana będzie trasa najszybszą linią i nawet jeżeli wymusza ona na nas później przesiadkę, to dla wcześniejszych przystanków nie będziemy w stanie jej już zmienić.
- przy wyliczaniu kosztu uwzględniana jest zmiana linii, która jeżeli występuje oznacza doliczenie do kosztu 1440 minut, co powoduje, że w kolejce wcześniej rozważone zostaną wszystkie krawędzi nie powodujące przesiadki.
- celem ułatwienia sprawdzenia wystąpienia przesiadki, do elementów w kolejce priorytetowej dodana została aktualna linia, która na samym początku ustawiona jest jako -1 (linia nieistniejąca).
- przy dodawaniu pierwszego elementu kolejki priorytetowej koszt początkowy wynosi -1440, gdyż odliczam pierwszy wybór linii.


In [14]:
def dijkstra_transfers(nodes_dict, start_stop, end_stop, start_time):
    cost_dict={}
    path = []
    cost = -1440

    priority_queue = [(cost, '-1', start_time, nodes_dict[start_stop], path)]

    while priority_queue:
        (current_cost, current_line, current_arrival_time, current_node, current_path) = heapq.heappop(priority_queue)

        if(current_node == nodes_dict[end_stop]):
            path = current_path
            cost = current_cost
            break

        for edge in current_node.edges:
            
            if(current_path==[]):
                cost_dict[f'{edge.line}_{current_node.stop_name}']=0
            
            line_change = calculate_transfer(current_line, edge)

            next_node = edge.end_stop
            
            transfer_key = f'{edge.line}_{next_node.stop_name}'
            
            new_cost = sum_cost(current_cost, current_arrival_time, edge.departure_time, edge.arrival_time) + calculate_transfer_as_time(line_change)
            
            if(transfer_key not in cost_dict or new_cost<cost_dict[transfer_key]):
                cost_dict[transfer_key]=new_cost

                new_path = current_path.copy()
                new_path.append(edge)

                heapq.heappush(priority_queue, (new_cost, edge.line, edge.arrival_time, next_node, new_path))

    return path, cost



A* - przesiadki

Tak jak w przypadku kryterium czasu, algorytm różni się uwzględnieniem priorytetu w kolejce priorytetowej, który obliczany jest poprzez zsumowanie obliczonego kosztu i odległości przystanku nastepnego od końcowego przeliczonego na czas przejazdu w minutach.

In [15]:
def a_star_transfers(nodes_dict, start_stop, end_stop, start_time):
    cost_dict={}
    path = []
    cost = -1440

    priority_queue = [(cost, cost, '-1', start_time, nodes_dict[start_stop], path)]

    while priority_queue:
        (priority, current_cost, current_line, current_arrival_time, current_node, current_path) = heapq.heappop(priority_queue)

        if(current_node == nodes_dict[end_stop]):
            path = current_path
            cost = current_cost
            break

        for edge in current_node.edges:
            
            if(current_path==[]):
                cost_dict[f'{edge.line}_{current_node.stop_name}']=0
            
            line_change = calculate_transfer(current_line, edge)

            next_node = edge.end_stop
            
            transfer_key = f'{edge.line}_{next_node.stop_name}'
            
            new_cost = sum_cost(current_cost, current_arrival_time, edge.departure_time, edge.arrival_time) + calculate_transfer_as_time(line_change)
            
            if(transfer_key not in cost_dict or new_cost<cost_dict[transfer_key]):
                cost_dict[transfer_key]=new_cost
                priority = new_cost + calculate_dist_as_time(nodes_dict[next_node.stop_name], nodes_dict[end_stop])

                new_path = current_path.copy()
                new_path.append(edge)

                heapq.heappush(priority_queue, (priority, new_cost, edge.line, edge.arrival_time, next_node, new_path))

    return path, cost



# Testowanie

Funkcje weryfikujące działnie algorytmów

Testy działania algorytmów przeprowadzone zostaną korzystając z funkcji test_algorithms, która 10 razy uruchamia każdą z przekazanych funkcji, a następnie wypisuje ich średni czas wykonywania oraz rezultat. Pojedyncze wywołanie realizowane jest przez funkcję test_function, która zwraca wynik i czas pojedynczego wywołania metody. Wypisywanie trasy realizowane jest przez funckję print_path.


In [19]:
def test_function(func, *args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    execution_time = end_time - start_time
    return (result, execution_time)

In [20]:
def test_algorithms(graph, start_stop, end_stop, start_time, *algorithms, repetitions=10):
    results = {algorithm.__name__: [] for algorithm in algorithms}
    execution_times = {algorithm.__name__: 0 for algorithm in algorithms}
    
    for i in range(repetitions):
        for algorithm in algorithms:
            result, exec_time = test_function(algorithm, graph, start_stop, end_stop, start_time)
            execution_times[algorithm.__name__] += exec_time
            results[algorithm.__name__] = result
     
    for algorithm in algorithms:
        avg_time = execution_times[algorithm.__name__]/repetitions
        path, cost = results[algorithm.__name__]
        print(algorithm.__name__)
        print("Average execution time for", repetitions, "repetitions:", avg_time)
        print("Total cost:", cost)
        print("Path: ")
        print_path(path)
        print()
        print()
        

In [21]:
def print_path(path):
    current_line = None
    start_line_time = None
    end_line_time = None
    start_stop = None
    end_stop = None
    
    for edge in path:
        if current_line != edge.line:
            if current_line is not None:
                print(f"{start_stop.stop_name} - {end_stop.stop_name} | {start_line_time.strftime('%H:%M:%S')} - {end_line_time.strftime('%H:%M:%S')} | Line: {current_line}")
            current_line = edge.line
            start_line_time = edge.departure_time
            start_stop = edge.start_stop
            
        end_line_time = edge.arrival_time
        end_stop = edge.end_stop
    
    if current_line is not None:
        print(f"{start_stop.stop_name} - {end_stop.stop_name} | {start_line_time.strftime('%H:%M:%S')} - {end_line_time.strftime('%H:%M:%S')} | Line: {current_line}")
        

Testy

In [23]:
test_algorithms(graph, 'TRZEBNICKA', 'PIASTOWSKA', datetime.strptime('02:00:00', '%H:%M:%S'), dijkstra_time, a_star_time, dijkstra_transfers, a_star_transfers)

dijkstra_time
Average execution time for 10 repetitions: 0.36800010204315187
Total cost: 47.0
Path: 
TRZEBNICKA - DWORZEC GŁÓWNY | 02:11:00 - 02:23:00 | Line: 247
DWORZEC GŁÓWNY - GALERIA DOMINIKAŃSKA | 02:24:00 - 02:28:00 | Line: 244
GALERIA DOMINIKAŃSKA - OGRÓD BOTANICZNY | 02:38:00 - 02:43:00 | Line: 246
OGRÓD BOTANICZNY - PIASTOWSKA | 02:45:00 - 02:47:00 | Line: 259


a_star_time
Average execution time for 10 repetitions: 0.17334728240966796
Total cost: 47.0
Path: 
TRZEBNICKA - DWORZEC GŁÓWNY | 02:11:00 - 02:23:00 | Line: 247
DWORZEC GŁÓWNY - GALERIA DOMINIKAŃSKA | 02:24:00 - 02:28:00 | Line: 244
GALERIA DOMINIKAŃSKA - OGRÓD BOTANICZNY | 02:38:00 - 02:43:00 | Line: 246
OGRÓD BOTANICZNY - PIASTOWSKA | 02:45:00 - 02:47:00 | Line: 259


dijkstra_transfers
Average execution time for 10 repetitions: 1.4323546886444092
Total cost: 198.0
Path: 
TRZEBNICKA - PIASTOWSKA | 05:06:00 - 05:18:00 | Line: 1


a_star_transfers
Average execution time for 10 repetitions: 0.9793622255325317
Total cos

In [24]:
test_algorithms(graph, 'KWISKA', 'PL. GRUNWALDZKI', datetime.strptime('09:00:00', '%H:%M:%S'), dijkstra_time, a_star_time, dijkstra_transfers, a_star_transfers)

dijkstra_time
Average execution time for 10 repetitions: 1.7144853115081786
Total cost: 26.0
Path: 
KWISKA - DOLMED | 09:01:00 - 09:07:00 | Line: 20
DOLMED - PL. GRUNWALDZKI | 09:07:00 - 09:26:00 | Line: 13


a_star_time
Average execution time for 10 repetitions: 0.20473790168762207
Total cost: 26.0
Path: 
KWISKA - DOLMED | 09:01:00 - 09:07:00 | Line: 20
DOLMED - PL. GRUNWALDZKI | 09:07:00 - 09:26:00 | Line: 13


dijkstra_transfers
Average execution time for 10 repetitions: 3.012541317939758
Total cost: 31.0
Path: 
KWISKA - PL. GRUNWALDZKI | 09:06:00 - 09:31:00 | Line: 10


a_star_transfers
Average execution time for 10 repetitions: 0.9373103380203247
Total cost: 31.0
Path: 
KWISKA - PL. GRUNWALDZKI | 09:06:00 - 09:31:00 | Line: 10


In [25]:
test_algorithms(graph, 'MAGELLANA', 'DWORZEC AUTOBUSOWY', datetime.strptime('11:00:00', '%H:%M:%S'), dijkstra_time, a_star_time, dijkstra_transfers, a_star_transfers)

dijkstra_time
Average execution time for 10 repetitions: 0.9317850589752197
Total cost: 26.0
Path: 
MAGELLANA - PL. GRUNWALDZKI | 11:03:00 - 11:14:00 | Line: 115
PL. GRUNWALDZKI - DWORZEC GŁÓWNY | 11:14:00 - 11:23:00 | Line: 145
DWORZEC GŁÓWNY - DWORZEC AUTOBUSOWY | 11:23:00 - 11:26:00 | Line: 9


a_star_time
Average execution time for 10 repetitions: 0.13678936958312987
Total cost: 26.0
Path: 
MAGELLANA - PL. GRUNWALDZKI | 11:03:00 - 11:14:00 | Line: 115
PL. GRUNWALDZKI - DWORZEC GŁÓWNY | 11:14:00 - 11:23:00 | Line: 145
DWORZEC GŁÓWNY - DWORZEC AUTOBUSOWY | 11:23:00 - 11:26:00 | Line: 9


dijkstra_transfers
Average execution time for 10 repetitions: 2.4561169385910033
Total cost: 831.0
Path: 
MAGELLANA - DWORZEC AUTOBUSOWY | 00:26:00 - 00:51:00 | Line: 259


a_star_transfers
Average execution time for 10 repetitions: 2.531494379043579
Total cost: 831.0
Path: 
MAGELLANA - DWORZEC AUTOBUSOWY | 00:26:00 - 00:51:00 | Line: 259



# Modyfikacja 

W ramach modyfikacji wykonałem porównanie dokładności funkcji obliczającej heurystyke, w zależności od sposobu wyliczania odległości między przystankami. Wyniki zestawiłem również ze średnim czasem przejazdu między przystankami pojedynczą linią w środę o godzinie 9:00.

In [26]:
def check_time_as_distance(start_stop, end_stop):
    euc = euclidian_distance(start_stop, end_stop)*416.25
    man = manhattan_distance(start_stop, end_stop)*416.25
    
    print('Stops:', start_stop.stop_name, '-', end_stop.stop_name)
    print('Euclidian:', euc)
    print('Manhattan', man)

In [27]:
check_time_as_distance(graph['TRZEBNICKA'], graph['PIASTOWSKA'])

Stops: TRZEBNICKA - PIASTOWSKA
Euclidian: 11.40456835952207
Manhattan 15.467470926928343


In [28]:
check_time_as_distance(graph['TRAMWAJOWA'], graph['NIEDŹWIEDZIA'])

Stops: TRAMWAJOWA - NIEDŹWIEDZIA
Euclidian: 38.355318662628854
Manhattan 44.873762016670454


In [29]:
check_time_as_distance(graph['DWORZEC NADODRZE'], graph['DWORZEC AUTOBUSOWY'])

Stops: DWORZEC NADODRZE - DWORZEC AUTOBUSOWY
Euclidian: 11.472850486831023
Manhattan 12.101246164399564


In [30]:
check_time_as_distance(graph['NYSKA'], graph['NOWOWIEJSKA'])

Stops: NYSKA - NOWOWIEJSKA
Euclidian: 17.67954834099821
Manhattan 21.30942364453564


In [31]:
check_time_as_distance(graph['CEGLANA'], graph['ŁUŻYCKA'])

Stops: CEGLANA - ŁUŻYCKA
Euclidian: 43.933850579156406
Manhattan 49.73111243638494


In [32]:
check_time_as_distance(graph['LEŚNICA'], graph['OPORÓW'])

Stops: LEŚNICA - OPORÓW
Euclidian: 48.60228889279903
Manhattan 66.78679861029849


In [33]:
check_time_as_distance(graph['PL. GRUNWALDZKI'], graph['PL. JANA PAWŁA II'])

Stops: PL. GRUNWALDZKI - PL. JANA PAWŁA II
Euclidian: 18.473351377546248
Manhattan 18.738214284453996


In [34]:
check_time_as_distance(graph['POŚWIĘTNE'], graph['KĘPIŃSKA'])

Stops: POŚWIĘTNE - KĘPIŃSKA
Euclidian: 2.964104345326998
Manhattan 3.6909054000022845


In [35]:
check_time_as_distance(graph['RYNEK'], graph['ŚWIDNICKA'])

Stops: RYNEK - ŚWIDNICKA
Euclidian: 2.792377156266145
Manhattan 3.882881280701387


In [36]:
check_time_as_distance(graph['DWORZEC GŁÓWNY'], graph['PORT LOTNICZY'])

Stops: DWORZEC GŁÓWNY - PORT LOTNICZY
Euclidian: 64.14980842935064
Manhattan 68.1981537060105


| PRZYSTANKI                            | EUCLIDIAN | MANHATTAN | CZAS RZECZYWISTY |
|---------------------------------------|-----------|-----------|------------------|
| TRZEBNICKA - PIASTOWSKA               | 11.4      | 15.5      | 12.0             |
| TRAMWAJOWA - NIEDŹWIEDZIA             | 38.4      | 44.9      | 30.0             |
| DWORZEC NADODRZE - DWORZEC AUTOBUSOWY | 11.5      | 12.1      | 18.0             |
| NYSKA - NOWOWIEJSKA                   | 17.7      | 21.3      | 25.0             |
| CEGLANA - ŁUŻYCKA                     | 43.9      | 49.7      | 33.0             |
| LEŚNICA - OPORÓW                      | 48.6      | 66.8      | 60.0             |
| PL. GRUNWALDZKI - PL. JANA PAWŁA II   | 18.5      | 18.7      | 15.0             |
| POŚWIĘTNE - KĘPIŃSKA                  | 3.0       | 3.7       | 2.0              |
| RYNEK - ŚWIDNICKA                     | 2.8       | 3.9       | 3.0              |
| DWORZEC GŁÓWNY - PORT LOTNICZY        | 64.1      | 68.2      | 38.0             |


Bazując na powyższych wynikach uznałem metodę euklidesową jako bardziej trafną dla mojego programu. Wynik heurystyki dla tego sposobu obliczania odległości był bliższy rzeczywistemu czasowi przejazdu w 7 z 10 przypadków. 

# Wnioski

Na początku największym wyzwaniem było wymyślenie przejrzystej i łatwej w funkcjonowaniu struktury, do której wczytywane będą dane. Kolejnym napotkanym problemem było przekazywanie i dostęp do danych w poszczegónych krokach algorytmów. Przy implementacji algorytmów związanych z kryterium czasu nie napotkałem większych kłopotów. Największym wyzwaniem okazała się implmentacja kryterium przesiadek, a konkretniej zauważenie potrzeby rozszerzenia klucza w słowniku kosztu o numer linii oraz ustawienie kosztu dla wszytskich linii przystanku początkowego na 0. Kolejna komplikacja, z którą musiałem się zmierzyć nastąpiła przy testowaniu czasów. Przez długi czas algorytmy a* potrafiły trwać wielokrotnie dłużej niż dijkstra, co według założeń powinno być niemożliwe. Po wielu próbach modyfikacji kodu, spostrzegłem błąd w funkcjach uśredniających lokalizaję przystanku podczas wczytywania danych, co powodowało błędnie wyliczane priorytety, które wydłużały czas działania funkcji.

Zgodnie z oczekiwaniami algorytm a* okazał się szybszy od algorytmu dijkstry. Jednakże różnica w czasie ich wykonywania zależna jest od rozważanego przypadku, co pokazują powyższe testy. Kluczową rolę w przyśpieszeniu czasu wykonywania funkcji ma heurystyka, która nieodpowienio dobrana może nawet spowodować sytuację, gdzie a* trwać będzie dłużej od algorytmu dijkstry.